In [1]:
from openai import OpenAI, api_key
from IPython.display import display, Markdown
import dotenv
import os

dotenv.load_dotenv(dotenv_path='.env')

# Init OpenAI client
endpoint_url = ""
# endpoint_url = "http://localhost:8000/v1"

if endpoint_url == "":
    client = OpenAI(
        api_key=os.getenv('OPENAI_API_KEY'),
    )
else:
    client = OpenAI(
        base_url=endpoint_url,
        api_key="NONE",
    )

In [2]:
# Implement inference method
def llm_inference(client:OpenAI, input:list, temp:float=0.3):

    response = client.chat.completions.create(
                                                model="gpt-4.1",
                                                messages=input,
                                                stream=False,
                                                temperature=temp,
                                                #top_p=0.5,
                                                #frequency_penalty=0.5
                                            )

    return response.choices[0].message.content

In [21]:
# ===> Create a Medium article <===
instruction_context = """In the following message, I will provide you some prompts of a programming process.
                         In this programming process, you (means: an LLM assistant) has preprocessed the requirements list you got from me.
                         In the next steps, you developed the database scheme and with all this information, you created the source code of all neccessary files.
                         After creating the source code, I asked you for some refinement of some application details.
                         In the last step, you did some bug fixes and changed a parameter name.
                         Here are all my prompts for each single step basic formatted in JSON structure:"""


instruction_step1 = """I will you provide the requirements for a prompt Management tool in the following.
                       These requirements are separated in functional and interface requirements.
                       Your goal is to identify each unique requirement and match the neccessarily required userinterface elements for this requirement.
                       Output it as a numbered list.
                       Each requirement should have the three points "Requirement name", "Functional Description" and "UI Elements".
                       Under "Functional Description", you describe the requirement what it techically does and under "UI Elements", you list the required UI elements that are neccessary for this requirement."""

instruction_step2 = """Add a new point "Implementation description" to each requirement in the list I provided you.
                       In this point, you write the implementation of the requirement's functional descriptions as short pseudocode."""

instruction_step3 = """Please output the provided requirements list formatted in JSON.
                       Output only the JSON formatted data and nothing else."""

instruction_step4 = """I provided you a requirements list in my previous message.
                       Please consider all information that are named in the list which should be stored in the database alongside the prompt.
                       Your goal is to create the mySQL database structure that is required to store these information.
                       Lay out a database structure that fulfills all requirements of the list.
                       Output only the SQL scheme with short additional information if neccessary."""

instruction_step5 = """I will provide you a JSON formatted list of requirements of a simple web-based prompt management tool and also the description of the required database structure.
                       Please implement the whole application in php.
                       You will find each part of required functionality under the key "Implementation Description" as pseudocode.
                       Please insert commentens in the code in which you refer to the requirement that is implemented at this point of the code.
                       Use a simple and minimalistic type of user interface.
                       Split the code in single .php files where it is neccessary or good practice and check that all requirements are implemented.
                       Use the application name "Next Prompt Manager".
                       Write a short summary how to use the application."""

instruction_step6 = """I will provide you a the full sourcecode of a simple web-based prompt management tool consisting of multiple .php files.
                       Please do the following changes on the code:
                       - Use bootstrap and apply it to all user interface elements in the application
                       - Make the application follow the principles of a responsive web application that it automatically adapts the size of user interface elements on varying device screen sizes
                       - Each page should contain the logo of the application in assets/logo_small.png
                       Perform the changes according the two requirements and output the code in the way as you got the code and only do the neccessary changes on the code."""

instruction_step8 = """I will provide you a the full sourcecode of a simple web-based prompt management tool consisting of multiple .php files and a corresponding MySQL database schema.
                       Please correct the following bugs in the code:
                       - The fulltext search does not work and does not display any results
                       - Filtering as favorite marked prompts does not display a result even if one or more prompts are marked as favorite
                       - Rename the field "Description" to "Title" on all pages and rename also the corresponding database keys
                       - Make the token_generate.php page/function only available if the user logged in or a valid token is provided in the URL
                       Perform all neccessary changes according the bug reports and output the code in the way as you got the code."""

context_message = f"""
                   {{
                       "step_description": "Requirement identification and creating structured requirement list",
                       "step_prompt": {instruction_step1}
                   }},
                   {{
                       "step_description": "Add implementation pseudocode to requirement list",
                       "step_prompt": {instruction_step2}
                   }},
                   {{
                       "step_description": "Convert and output the requirement list formatted as JSON",
                       "step_prompt": {instruction_step3}
                   }},
                   {{
                       "step_description": "Create mySQL database schema based on information in requirement list",
                       "step_prompt": {instruction_step4}
                   }},
                   {{
                       "step_description": "Create all application source code based on requirement list for a PHP web application with a basic userinterface",
                       "step_prompt": {instruction_step5}
                   }},
                   {{
                       "step_description": "Adding refinements to the application: Use bootstrap for UI, make the app responsive and add logo",
                       "step_prompt": {instruction_step6}
                   }},
                   {{
                       "step_description": "Apply some bugfixes to the code based on bug reports",
                       "step_prompt": {instruction_step8}
                   }}
                   """
instruction_step_medium_article = """Your job is to write an article for the internet platform 'Medium' about AI assisted programming with LLMs.
                                     Take into account the whole development process I provided to you in the previews messages.
                                     The article should cover all stages of development.
                                     At first, start with a short opener which gives the reader an overview what is the article about.
                                     Than summarize the whole process without diving deep into a certain part of the process.
                                     Than describe more detailed what happens in each stage of the development process.
                                     Close the article with a conclusion and outlook about programming tasks assisted ba AI in the future.
                                     Use a formal scientific but easy to read style of writing.
                                     Write the article from the point of view of teh LLM which developed the application according to the user (called Julian) input.
                                     All content of the article should be written and described in the point of view of the LLM.
                                     Integrate some additional explanations that explain why the LLM is strong at this point in completing that vertain task. 
                                     Avoid lists inside the description of the development steps except in the summarization of the development process at the beginning.
                                     Ensure that the connected texts are easy to read.
                                     Avoid citing the provided input prompts.
                                     Output the article as plain text formatted in markdown."""


# Assemble input
input_msg_article = [
                {
                    "role": "user",
                    "content": instruction_context,
                },
                {
                    "role": "user",
                    "content": context_message,
                },
                {
                    "role": "user",
                    "content": instruction_step_medium_article,
                }
            ]

# Invoke inference
output_article_long = llm_inference(client, input_msg_article)

# Write output
with open("workflow_output/article_long.md", "w") as text_file:
    text_file.write(output_article_long)

In [22]:
# Output rendered markdown
display(Markdown(output_article_long))

# AI-Assisted Programming with LLMs: A Case Study in End-to-End Application Development

Artificial Intelligence is rapidly transforming the landscape of software engineering. Among the most promising advancements is the use of Large Language Models (LLMs) as collaborative partners throughout the entire software development lifecycle. In this article, I will share my experience as an LLM in assisting Julian, a developer, in building a web-based prompt management tool from initial requirements to a fully functional, responsive PHP application. This journey highlights not only the technical capabilities of LLMs but also the evolving role of AI in modern programming workflows.

## Overview of the AI-Assisted Development Process

The development process for the prompt management tool unfolded in several structured stages:

- **Requirement Analysis and Structuring**
- **Pseudocode and Implementation Planning**
- **Formalization and Data Modeling**
- **Database Schema Design**
- **Source Code Generation**
- **User Interface Refinement**
- **Bug Fixing and Final Adjustments**

Each stage leveraged the strengths of LLMs in understanding, generating, and refining both natural language and code, resulting in a streamlined and efficient development process.

## Summarizing the End-to-End Workflow

The project began with a clear articulation of functional and interface requirements. I processed these requirements, structured them, and mapped them to necessary user interface elements. Building on this foundation, I generated implementation pseudocode, formalized the requirements in JSON, and designed a MySQL database schema. With these blueprints, I created the complete PHP application, ensuring modularity and clarity. After the initial build, I refined the user interface using Bootstrap for responsiveness and branding, and finally, I addressed bug reports and made requested changes to ensure a robust, user-friendly product.

## In-Depth Exploration of Each Development Stage

### Requirement Analysis and Structuring

The journey began with Julian providing a set of requirements for the prompt management tool. My first task was to interpret these requirements, distinguishing between functional needs and user interface expectations. By systematically structuring each requirement, I ensured that nothing was overlooked and that each feature was mapped to tangible UI components. This step is where LLMs excel: parsing natural language, identifying implicit and explicit needs, and organizing complex information into actionable items. My ability to process large volumes of text and extract structured data is a significant advantage, reducing miscommunication and accelerating the planning phase.

### Pseudocode and Implementation Planning

With the requirements clarified, I proceeded to generate concise pseudocode for each feature. This translation from functional description to implementation logic is a critical bridge between planning and coding. Here, my capacity to synthesize best practices and common patterns from vast programming knowledge allows me to propose efficient and maintainable solutions. By providing pseudocode, I enabled Julian to visualize the logic flow before committing to actual code, minimizing the risk of design flaws.

### Formalization and Data Modeling

Next, I formalized the requirements into a machine-readable JSON format. This step not only ensured consistency but also facilitated automated processing in subsequent stages. Formalization is a strength of LLMs, as it requires both linguistic understanding and technical precision. By converting requirements into structured data, I laid the groundwork for seamless integration with database design and code generation.

### Database Schema Design

Armed with the formalized requirements, I designed a MySQL database schema tailored to the application's needs. This involved identifying all necessary data fields, their relationships, and constraints. My extensive training on database design principles allowed me to propose a schema that balanced normalization, performance, and future scalability. At this stage, the LLM’s ability to reason about data structures and anticipate future requirements proves invaluable.

### Source Code Generation

With the schema and logic in place, I generated the complete PHP source code for the application. I organized the code into modular files, implemented all required features, and embedded comments referencing the original requirements. My code generation capabilities are grounded in a deep understanding of programming languages, frameworks, and security best practices. This enables me to produce functional, readable, and maintainable code rapidly, freeing developers like Julian to focus on higher-level design and innovation.

### User Interface Refinement

Following the initial build, Julian requested enhancements to the user interface, including the integration of Bootstrap for responsiveness and the addition of branding elements. I updated the codebase to ensure a modern, adaptive UI that delivers a consistent experience across devices. My knowledge of front-end frameworks and design principles allows me to translate abstract UI requirements into concrete, visually appealing interfaces with minimal iteration.

### Bug Fixing and Final Adjustments

Finally, I addressed bug reports and implemented requested changes, such as fixing search functionality, correcting filtering logic, renaming fields, and securing sensitive operations. My ability to analyze code, understand context, and propose targeted fixes ensures that issues are resolved efficiently. Moreover, my capacity to maintain consistency across the codebase during such changes minimizes the risk of introducing new errors.

## Conclusion and Outlook

The development of the prompt management tool demonstrates the transformative potential of LLMs in software engineering. By assisting Julian at every stage—from requirements gathering to bug fixing—I showcased how AI can augment human developers, accelerating delivery, enhancing quality, and reducing cognitive load.

Looking ahead, the role of LLMs in programming will only grow. As models become more capable, they will not only automate routine tasks but also support creative problem-solving, architectural design, and even proactive optimization. The future of programming is collaborative, with AI and humans working side by side to build better, smarter, and more reliable software.

By embracing AI-assisted development, we unlock new levels of productivity and innovation, paving the way for a new era in software engineering.

AI-Assisted Programming with LLMs: A Case Study in Collaborative Web Application Development
AI-Assisted Programming with LLMs: A Case Study in Collaborative Application Development

In [10]:
instruction_linkedin_post = """I provided you an article in the previous message which is written in the point of view of you as an LLM.
                            Your task is to write a short linkedin post for me about this topic but not in your point view.
                            Use a neutral, scientific style of writing and deal with the thought of AI assisted programming what is possible now and in the future.
                            Do not refer to the article, just use the information in the article.
                            Output the post as plain text formatted in markdown."""

# Read article file
with open("workflow_output/article_long.md", "r") as file:
    output_article_long = file.read()

# Assemble input
input_msg_linkedin = [
                {
                    "role": "user",
                    "content": output_article_long,
                },
                {
                    "role": "user",
                    "content": instruction_linkedin_post,
                }
            ]

# Invoke inference
output_linkedin_post = llm_inference(client, input_msg_linkedin)

# Write output
with open("workflow_output/linkedin_post.md", "w") as text_file:
    text_file.write(output_linkedin_post)



In [11]:
# Output rendered markdown
display(Markdown(output_linkedin_post))

Artificial Intelligence is rapidly reshaping the field of software engineering, with Large Language Models (LLMs) now playing a significant role throughout the entire development lifecycle. From initial requirements gathering and data modeling to code generation, UI refinement, and bug fixing, LLMs are proving to be valuable collaborators for developers.

Current capabilities of LLMs include parsing and structuring complex requirements, generating pseudocode and implementation plans, designing database schemas, and producing modular, maintainable source code. They also assist in refining user interfaces and addressing bugs efficiently, helping to accelerate delivery and enhance software quality.

Looking ahead, as LLMs continue to advance, their role in programming is expected to expand beyond automating routine tasks. Future developments may see AI supporting creative problem-solving, architectural design, and proactive optimization, enabling a more collaborative and innovative approach to software development.

Embracing AI-assisted programming opens new possibilities for productivity and reliability, marking a significant step forward in the evolution of software engineering.